In [1]:
import pandas as pd
import os
import requests
import json

ontoserver_host = "101.43.110.249:8443"
ontoserver_base_url = f"https://{ontoserver_host}/fhir"

df = pd.read_excel("../data samples/Indications (AMS)_Australia_19.xlsx")

In [2]:
import os

class ValueSet:

    def __init__(self, base_url, version, name, publisher, contact, description, status, experimental, values):
        self.resourceType = "ValueSet"
        self.url = os.path.join(base_url, self._get_name(name))
        self.version = version
        self.name = name
        self.publisher = publisher
        self.contact = [{"telecom":[{"system":k, "value":v} for k, v in contact.items()]}]
        self.description = description
        self.status = status
        self.experimental = experimental    # True or false
        self.compose = {
            "include": [
                {
                    "system": os.path.join(base_url, self._get_name(name)),
                    "concept": [{"code":v} for v in values]
                }
            ]
        }
    
    def to_json(self):
        data = {}
        for key in self.__dict__:
            value = getattr(self, key)
            data[key] = value
        return data

    def _get_name(self, name):
        splited_name = name.lower().split(' ')
        return '-'.join(splited_name)


In [9]:
import requests
import os
from datetime import datetime

def create_value_set(url, name, status, experimental, publisher, contact, description, values):
    
    version = datetime.now().strftime(f"%Y%m%d%H%M%S")

    value_set = ValueSet(ontoserver_base_url, 
                    version, 
                    name, 
                    publisher, 
                    contact, 
                    description, 
                    status, 
                    experimental, 
                    values)

    #  requeststo create new value set
    response = requests.post(os.path.join(ontoserver_base_url, "ValueSet"),  json=value_set.to_json(), headers={"Content-Type": "application/fhir+json"}, verify=False)
    return response

response = create_value_set(ontoserver_base_url, name = "Universal Indication List", 
                    status="draft", experimental=True, publisher="Daniel", 
                    contact={"email":"test@test.com"}, 
                    description="Indications", values=df["Indication"].tolist())

print(response.json())



{'resourceType': 'ValueSet', 'id': 'b602b9bf-41a2-43f6-8232-738dd77ca960', 'meta': {'versionId': '1', 'lastUpdated': '2023-04-07T01:20:10.951+00:00'}, 'url': 'https://101.43.110.249:8443/fhir/universal-indication-list', 'version': '20230407092010', 'name': 'Universal Indication List', 'status': 'draft', 'experimental': True, 'publisher': 'Daniel', 'contact': [{'telecom': [{'system': 'email', 'value': 'test@test.com'}]}], 'description': 'Indications', 'compose': {'include': [{'system': 'https://101.43.110.249:8443/fhir/universal-indication-list', 'concept': [{'code': 'Compound or open fracture'}, {'code': 'Infected prosthetic joint or bone prosthesis'}, {'code': 'Osteomyelitis'}, {'code': "Reactive arthritis (Reiter's disease)"}, {'code': 'Septic arthritis'}, {'code': 'Septic bursitis'}, {'code': 'Infective tenosynovitis'}, {'code': 'Vertebral osteomyelitis'}, {'code': 'Bacterial meningitis, community acquired, pathogen unknown'}, {'code': 'Bacterial meningitis / ventriculitis, healthca

/Users/sunkunxi/miniforge3/envs/ml/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '101.43.110.249'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:
import requests

